In [306]:
from Bio import SeqIO
import numpy as np
import torch.nn as nn
import torch
cos = nn.CosineSimilarity(dim=0, eps=1e-6)
import torch.nn.functional as F
from Bio import AlignIO
import pandas as pd
import torch
import esm
import matplotlib.pyplot as plt
import seaborn as sns
from Bio.PDB import PDBParser, PPBuilder
import warnings
from Bio.PDB.PDBExceptions import PDBConstructionWarning
from Bio.SeqUtils import IUPACData
from collections import defaultdict
import os
import sys
import urllib.request
import Bio.PDB
import random
from collections import Counter

In [173]:
base_dir = '/Users/williamharrigan/Desktop/Github/contact_site_classifier/attention_classifier/data_files/'
fasta_file = base_dir + 'rcsb_pdb_3KYN.fasta'
pdb_filename = base_dir + '3kyn.pdb'
structure_dir = base_dir +'structure_files/'
fasta_dir = base_dir +'fasta_files/'
casp = base_dir + 'casp7/'

In [ ]:
## This code is not important right now, will be when adding multiple sequences

# seqs = SeqIO.to_dict(SeqIO.parse(full_len_sequences, "fasta"))
# for k,v in seqs.items():
#     seqs[k] = str(v.seq)
    
# print(list(seqs.keys())[:5])

In [3]:
def simple_aa(three_letter_code):
    return IUPACData.protein_letters_3to1.get(three_letter_code.capitalize())

## To Do:


1. Get sequence IDs from CASP training file
2. Extract sequence from sequence ID from PDB database
3. Extract .pdb structural file from PDB for same ID
4. Double check that seqeunce files that are extracted are the same as the structural files
5. Get embeddings from extracted sequences (try 100 sequences first)
6. Turn contact site code into functions that can be run iteratively
7. Integrate this code with the classifier
    1. Make sure that when training model we balance positive and negative cases. (Start with 50/50 balance)
    2. Do so by making contact and non-contact dictionaries to pull training data from
8. Start working on implementing statstics like AUROC, loss functions and other hyperparameter optimization tasks. 

## Extract Sequence IDs and Structure (.pdb) files

In [4]:
def parse_casp7_file(file_path):
    data_dict = {}

    with open(file_path, 'r') as file:
        for line in file:
            line = line.strip()
            if line.startswith('[ID]'):
                sequence_id = next(file).strip()
            elif line.startswith('[PRIMARY]'):
                sequence = next(file).strip()
#                 print(sequence_id, sequence)
                data_dict[sequence_id] = sequence

    return data_dict

In [5]:
def download_pdb(pdb_id, structure_dir, downloadurl="http://files.rcsb.org/download/"):
    """
    Downloads a PDB file from the Internet and saves it in a data directory.
    :param pdbcode: The standard PDB ID e.g. '3ICB' or '3icb'
    :param datadir: The directory where the downloaded file will be saved
    :param downloadurl: The base PDB download URL, cf.
        `https://www.rcsb.org/pages/download/http#structures` for details
        Note that the unencrypted HTTP protocol is used by default
        to avoid spurious OpenSSL errors...
    :return: the full path to the downloaded PDB file or None if something went wrong
    """
    pdbfn = pdb_id + ".pdb"
    url = downloadurl + pdbfn
    outfnm = os.path.join(structure_dir, pdbfn)
    try:
        urllib.request.urlretrieve(url, outfnm)
        return pdbfn
    except Exception as err:
        # all sorts of things could have gone wrong...
        print(str(err), file=sys.stderr)
        return None

In [178]:
def download_fasta(pdb_id, fasta_dir, downloadurl="https://www.rcsb.org/fasta/entry/"):
    pdbfn = pdb_id
    url = downloadurl + pdbfn
    outfnm = os.path.join(fasta_dir, F'{pdbfn}.fasta')
    try:
        urllib.request.urlretrieve(url, outfnm)
        return pdbfn
    except Exception as err:
        # all sorts of things could have gone wrong...
        print(str(err), file=sys.stderr)
        return None

In [204]:
### HERE
# Turned off warnings for discontinuous data structures

warnings.simplefilter('ignore', PDBConstructionWarning)

# Load the structure from locally saved file
parser = PDBParser()

In [271]:
def check_sequences(pdb_id):
    # Parse pdb file and save as structure. The pdb file is where we are getting CA coordinates from.
    structure = parser.get_structure(pdb_id, f"{structure_dir+pdb_id}.pdb")

    # Extract desired protein structure from PDB structure (typically only 1 structure to choose from)
    protein_structure = structure[0]

    residue_position = 0
    mismatches = 0
    print(pdb_id)
    if 'A' in protein_structure:
        for residue in protein_structure['A']:
            if 'CA' in residue:
                if residue_position < len(protein_data[pdb_id]):
                    if simple_aa(residue.resname) != protein_data[pdb_id][residue_position]:
#                         print(residue.id[1], simple_aa(residue.resname), protein_data[pdb_id][residue_position])
                        mismatches+=1
            residue_position+=1
        if mismatches == 0:
            same_sequence_ids.append(pdb_id)
#         print(pdb_id)
    return 


In [ ]:
# 2. Prepare sequence input from a fasta file and already loaded .pdb file
# Load sequence file

# seq_chains = []

# for record in SeqIO.parse(fasta_file, "fasta"):
#     seq_chains.append(str(record.seq))

# pdb_id = record.id.split('|')[0].split('_')[0]
# protein_sequence = ''.join(seq_chains)
# Replace 'training_30.txt' with the path to your file

In [181]:
# Get sequences from CASP7 file
prot_data_dict = parse_casp7_file(casp+'training_95')

protein_id_counts = Counter(protein_id.split('_')[0] for protein_id in prot_data_dict)

# Now, create the test list with only those protein_ids that occur exactly once
single_occurence_ids = [protein_id for protein_id, count in protein_id_counts.items() if count == 1]

In [253]:
download_fasta('1TKI', fasta_dir) == None

False

In [256]:
# Load sequence file

# protein_data = {}

for pdb_id in single_occurence_ids[2681:]:
    check = download_fasta(pdb_id, fasta_dir)
    if check == None:
        continue
    else:
        download_pdb(pdb_id, structure_dir)
        fasta_file = fasta_dir + pdb_id + '.fasta'
        for record in SeqIO.parse(fasta_file, "fasta"):
            print(record.id.split('|')[0].split('_')[0])
            print(len(str(record.seq)))
            protein_data[record.id.split('|')[0].split('_')[0]] = str(record.seq)
            break

# pdb_id = record.id.split('|')[0].split('_')[0]
# protein_sequence = ''.join(seq_chains)
# Replace 'training_30.txt' with the path to your file

1YBD
239
1PSM
38
1DM0
287
1WFO
130
1YOA
159
1Q67
231
1SHZ
340
1WHY
97
1B67
68
1VMK
277
1YQS
349
1M8A
70
1WY8
89
1VLM
219
1PDO
135
1Q01
38
2CVK
110
1NGQ
215
2CW1
65
1ISP
181
1W94
156
1ULS
245
1Y2Q
143
2CVB
188
1Z9V
121
1JQK
639
1WWZ
159
1HFO
113
2COD
115
1ZYP
202
2FQX
318
2COW
100
1S28
132
1UMG
362
2AIE
203
1OYG
447
1G4I
123
1IUA
83
1MUG
168
2CDE
192
1HS7
97
1R75
141
1N7H
381
1ZN8
180
1UDC
338
1SRK
35
2CCF
34
1KZH
555
1R7L
110
1EMX
30
1MGP
313
1V5P
126
1IIB
106
1RW1
114
1MIQ
375
1RM8
169
1KZ1
159
2FIA
162
2GHS
326
1VIA
175
1QXN
137
2BAP
317
1MC7
95
1I74
309
1CXC
124
2FVL
324
1PD6
104
2AR0
541
1WQJ
80
1YUJ
11
1SUM
235
1UWU
489
2F2B
246
1TXC
157
1L1N
183
1DUM
23
1GY8
397
1VIZ
240
1NI8
46
1YJV
75
1AY7
96
1WYN
146
1WQB
32
1SXH
280
7R1R
761
1AAP
58
2ARF
165
2F9H
129
1TET
216
1HQ1
49
2AHX
617
2FG9
178
2FR8
384
2BGP
140
1UST
93
1QWG
251
1WH2
78
1P9M
299
2MYO
118
1TZK
338
1MZR
296
2AEG
268
2CH7
309
1MD6
154
1TRE
255
1LVO
302
2BCW
65
2FZF
175
1QEY
31
1QU7
227
2F15
96
1KL8
74
2E2A
105
1WIS
124
1X

HTTP Error 404: Not Found


1C3P
375
1YG6
193
1UHF
69
1DFO
417
1IPG
85
1MNA
298
1FSL
143
1R5Q
102
1UWC
261
1K68
140
2MCM
112
2D2E
250
1YDF
257
1BNB
38
4OTC
62
2BHM
164
1M8S
124
1QXH
167
1WOC
103
1ZHX
438
1EE6
197
1OP4
159
1YLL
200
1YR0
175
1S6U
76
201L
166
1BI9
499
1WYS
75
1YRL
491
1HQL
257
1NZ8
119
1JAC
133
1S4V
229
1BO4
168
2BZG
232
1B9O
123
1XG7
250
1XW3
110
1VHH
162
1J0W
103
1PZM
211
1ET1
34
1VZW
244
1I6Z
135
1QTF
246
2CTJ
95
1V4B
200
1Q0E
152
1Z14
549
1QC7
101
2F8J
347
1V8Y
170
1IXL
131
2HVM
273
2BH4
134
1K55
246
2A7M
250
1C7C
283
1SEV
362
1ANS
27
1C44
123
1RDG
52
1K24
253
1PV1
299
1KGN
329
1BLE
163
1RY2
663
1V7L
163
1G2T
71
1S6W
21
2CP6
172
1KHV
516
1SNB
64
1VTX
42
1QZU
206
1KVJ
79
2AS8
222
1XKW
665
1NFF
260
1F07
321
1XNT
183
1ZSO
164
2C5A
379
1HS5
34
1V5K
115
2AIB
98
1ARK
60
1Y2T
142
1XI4
1630
1FKQ
124
1JPY
137
1N7K
234
1SJR
164
1B9Q
34
1G2Q
187
1OL0
121
2FHL
122
1M7B
184
1JI0
240
1WLM
151
1J02
267
2AU5
139
1E5W
346
1RGY
360
1WFX
186
1G2L
235
1QU9
128
1QPG
415
1UAY
242
1JJI
311
1U7I
136
1YBK
52
1ZR0
63
1MR

1GNY
153
1HU3
260
2C0H
353
1X2P
68
2LIS
136
1GZS
180
1PTF
88
1QP3
70
1RFL
172
1NTC
91
1XRI
151
1WXL
73
1U2E
289
1K4M
213
1NXJ
183
2FLI
220
1NCX
162
2CUF
95
1TOT
52
2B4G
317
8PCH
220
2B7T
73
1I96
1514
1TH0
226
1I12
160
1XFU
747
1BGC
174
2FH9
274
1WYJ
125
1WYR
121
1TJW
474
1LSS
140
1DZ3
130
2MPR
427
1VJQ
79
1EIY
76
1TSJ
139
1P3C
215
1FGL
165
1NNF
309
1WHV
100
1LP3
519
1M4U
139
1BHE
376
1SC0
138
1QE3
489
2BN5
33
1W41
101
1YK3
210
1F2D
341
1HKU
358
1X1N
524
1NF3
195
1P1A
85
1Y7K
53
1PR9
244
1HXR
115
1XE7
203
1SB6
64
1I4Y
114
1XWH
66
1B6Z
140
1DL7
109
1N6C
297
2COE
120
1HE1
135
1DV8
128
1UXV
501
1QQE
292
1AO5
237
1DS1
324
1OR5
83
1UJT
120
1R4T
153
1QE5
266
1WBA
175
1UR3
319
1J1Q
261
1NG0
253
1ZH2
121
1ZU4
320
1R03
182
1QO2
241
1X87
551
1NR9
223
1F2L
76
3KVT
115
2HP8
79
2ACA
189
1NR4
71
1QNX
209
2A5L
200
1FE4
68
1I8O
114
1WUW
45
1AGG
48
2AMY
246
1YPV
313
1N6Z
105
2AXJ
242
1PML
86
1S1G
124
1A6J
163
1XVX
312
1OWC
427
1Y4I
398
1IVZ
132
1E29
135
1GY7
125
1G7N
131
2GL8
241
1RVG
305
2HCC
66
1ADW
1

1WRA
308
1U5L
108
1ZK4
251
2CFQ
417
1BXY
60
1THM
279
1TWO
142
2F8Y
223
2BEM
170
1XOK
30
1ZUP
315
1A75
108
1K0K
125
1HK8
605
1CIX
44
1VMH
144
2COF
107
1BS0
384
1RL1
114
1E87
118
1L6R
227
1IA9
280
1RTL
200
1HJ8
222
1GKD
163
1R7G
31
2BT2
161
1DUS
194
1JNT
92
1AH9
71
1LLN
262
1L5P
93
2FNF
72
1YZQ
170
1SH7
284
1WHA
105
1NXC
478
1OV9
50
1BY2
119
2BKF
87
1VKW
218
1ZKI
133
2EZE
12
1WH7
80
1IF1
26
2B9E
309
1F45
306
2CRP
150
1SQH
312
1Z5Z
271
1EDV
34
1CKA
57
1X8D
104
1KEH
689
1RDU
116
1CT5
256
1WHX
111
1HCD
118
1WF5
121
1A0J
223
2A4C
99
1CHK
238
1E57
188
1KV5
250
1A92
50
1WWN
69
1CQK
101
1SR2
116
2CPH
107
1IDR
136
1DDL
7
1X7V
99
1UUN
184
1I16
130
1ATL
202
1VMD
178
1HY5
136
1MRE
219
1YUC
255
1ANP
28
1V1C
71
1KSF
758
1NCG
110
1FCH
368
1LY1
181
1HAR
216
2BF9
36
1A8Q
274
2FV8
207
1IIJ
35
2FUP
157
1LNW
147
1P28
129
1RF6
427
1MBA
147
1PKU
150
1SHX
138
2FT6
124
1IM3
275
1XC0
33
1BXS
501
1Y97
238
1Q27
171
2C5R
67
1G0S
209
1XYN
178
1UGV
72
1DLI
402
1W9T
142
2G2C
167
1YQD
366
1SMP
471
1PFS
78
1W1H
151
3HF

KeyboardInterrupt: 

In [257]:
len(protein_data)

2681

In [275]:
# Prepare sequence input to ESM/contact analysis from CASP7 dataset

# Load CASP file containing PDB IDs and Sequences
# protein_data = parse_casp7_file(base_dir+'training_30')
# same_sequence_ids = []

# for pdb_id, protein_sequence in list(protein_data.items()):
#     print('Sequence ID: ', pdb_id)
# #     print('Sequence: ', protein_sequence, len(protein_sequence), '\n')
    
#     # Download pdb structure files from PDB database using CASP7 protein ids
#     print('PDB Structural File Output: ', download_pdb(pdb_id, structure_dir), '\n')
#     check_sequences(pdb_id)
    # Index protein sequence as sequence 0 (next sequence would be indexed as 1)
#     esm_input_data = [(0, protein_sequence)]
#     print('Data: ', esm_input_data, '\n')

#     # Prepare variables to input sequence into ESM-2 model 
#     batch_converter = alphabet.get_batch_converter()
#     batch_labels, batch_strs, batch_tokens = batch_converter(esm_input_data)
#     batch_tokens = batch_tokens.cuda() if torch.cuda.is_available() else batch_tokens
    
#     print('batch_tokens: ', '\n\n', batch_tokens, '\n')

In [274]:
len(same_sequence_ids)

1214

In [277]:
same_sequence_ids[:5]

['1PSM', '1WFO', '1YOA', '1WHY', '1B67']

Model outputs:  dict_keys(['logits', 'representations', 'attentions', 'contacts'])


## Calculating Expected Contact Sites from PDB structure file

In [225]:
# parser = PDBParser()
# structure = parser.get_structure(pdb_id, f"{structure_dir+pdb_id}.pdb")

# # Extract desired protein structure from PDB structure (typically only 1 structure to choose from)
# protein_structure = structure[0]


# for chain in protein_structure:
#     print("Chain:", chain.id)  # Reset position for each chain

#     for residue in chain:
# #         print(residue.id)
#         if residue_position >= len(protein_data[pdb_id]):
#             print("Sequence length exceeded at chain", chain.id)
#             break  # Break the loop if the position exceeds the sequence length
#         try:
#             if simple_aa(residue.resname) != protein_data[pdb_id][residue_position]:
#                 print("Mismatch at position:", residue.id[1])
#                 break
#             else:
#                 if simple_aa(residue.resname) == None:
#                     continue
#                 else:
#                     print(residue.id[1], simple_aa(residue.resname), protein_data[pdb_id][residue_position], residue['CA'])
#                 residue_position += 1
            
#         except KeyError:
#             # Specific exception handling
#             continue
#     print(residue_position)

In [322]:
def calc_contact_sites(pdb_id):
    structure = parser.get_structure(pdb_id, f"{structure_dir}/{pdb_id}.pdb")  # Ensure correct path joining
    protein_structure = structure[0]
    chain = protein_structure['A']

    # Initialize count variable
    count = 0

    for i, residue1 in enumerate(chain):
        for j, residue2 in enumerate(chain):
            if i <= j:
                continue  # Avoids redundant comparisons and self-comparison
            try:
                distance = abs(residue1['CA'] - residue2['CA'])
            except KeyError:
                continue
            if distance < 5:
                if abs(residue1.id[1] - residue2.id[1]) > 2:
    #                 print(residue1.id[1], residue1.resname, residue2.id[1], residue2.resname, distance)
                    in_contact_sites[pdb_id].append({
                        'res_1': residue1.id[1], 
                        'res_2': residue2.id[1], 
                        'sig_1': simple_aa(residue1.resname), 
                        'sig_2': simple_aa(residue2.resname), 
                        'dist': distance,
                        'in_contact': True
                    })
                    count += 1
            else:
                if abs(residue1.id[1] - residue2.id[1]) > 2:
                    non_contact_sites[pdb_id].append({
                        'res_1': residue1.id[1], 
                        'res_2': residue2.id[1], 
                        'sig_1': simple_aa(residue1.resname), 
                        'sig_2': simple_aa(residue2.resname), 
                        'dist': distance,
                        'in_contact': False
                    })

    if non_contact_sites[pdb_id]:
        subset_non_contact_sites[pdb_id] = random.sample(non_contact_sites[pdb_id], min(len(non_contact_sites[pdb_id]), len(in_contact_sites[pdb_id])))

    # Optionally print or process the results
    return f"Total contacts found {pdb_id}: {count}"

In [327]:
in_contact_sites = defaultdict(list)
non_contact_sites = defaultdict(list)
subset_non_contact_sites = defaultdict(list)

iterations = 0

for pdb_id in same_sequence_ids:
    print(calc_contact_sites(pdb_id))
    print(len(in_contact_sites[pdb_id]), len(subset_non_contact_sites[pdb_id]))
    print("Iterations: ", iterations)
    iterations+=1

Total contacts found 1PSM: 8
8 8
Iterations:  0
Total contacts found 1WFO: 40
40 40
Iterations:  1
Total contacts found 1YOA: 40
40 40
Iterations:  2
Total contacts found 1WHY: 22
22 22
Iterations:  3
Total contacts found 1B67: 8
8 8
Iterations:  4
Total contacts found 1WY8: 22
22 22
Iterations:  5
Total contacts found 1PDO: 28
28 28
Iterations:  6
Total contacts found 1Q01: 4
4 4
Iterations:  7
Total contacts found 2CW1: 21
21 21
Iterations:  8
Total contacts found 1Y2Q: 47
47 47
Iterations:  9
Total contacts found 1WWZ: 34
34 34
Iterations:  10
Total contacts found 1HFO: 28
28 28
Iterations:  11
Total contacts found 2COD: 24
24 24
Iterations:  12
Total contacts found 1ZYP: 50
50 50
Iterations:  13
Total contacts found 2COW: 19
19 19
Iterations:  14
Total contacts found 1G4I: 41
41 41
Iterations:  15
Total contacts found 1IUA: 22
22 22
Iterations:  16
Total contacts found 1MUG: 43
43 43
Iterations:  17
Total contacts found 2CDE: 60
60 60
Iterations:  18
Total contacts found 1HS7: 41
4

Total contacts found 1KLF: 61
61 61
Iterations:  158
Total contacts found 2C1T: 87
87 87
Iterations:  159
Total contacts found 1NKI: 30
30 30
Iterations:  160
Total contacts found 1UX0: 43
43 43
Iterations:  161
Total contacts found 1ZOK: 26
26 26
Iterations:  162
Total contacts found 1UFF: 19
19 19
Iterations:  163
Total contacts found 2BCK: 87
87 87
Iterations:  164
Total contacts found 2CP0: 24
24 24
Iterations:  165
Total contacts found 1X6D: 32
32 32
Iterations:  166
Total contacts found 1KAF: 29
29 29
Iterations:  167
Total contacts found 1H4P: 104
104 104
Iterations:  168
Total contacts found 1VEC: 46
46 46
Iterations:  169
Total contacts found 1BKT: 20
20 20
Iterations:  170
Total contacts found 1BTN: 30
30 30
Iterations:  171
Total contacts found 1W4I: 21
21 21
Iterations:  172
Total contacts found 1WJN: 21
21 21
Iterations:  173
Total contacts found 1PMH: 0
0 0
Iterations:  174
Total contacts found 2FDN: 9
9 9
Iterations:  175
Total contacts found 1V6F: 41
41 41
Iterations:  

Total contacts found 1WD2: 13
13 13
Iterations:  315
Total contacts found 1ZNV: 12
12 12
Iterations:  316
Total contacts found 1N4N: 12
12 12
Iterations:  317
Total contacts found 2B5S: 19
19 19
Iterations:  318
Total contacts found 1J32: 105
105 105
Iterations:  319
Total contacts found 2COK: 46
46 46
Iterations:  320
Total contacts found 1UGL: 17
17 17
Iterations:  321
Total contacts found 1WRM: 53
53 53
Iterations:  322
Total contacts found 1X3K: 28
28 28
Iterations:  323
Total contacts found 2LTN: 43
43 43
Iterations:  324
Total contacts found 1LIQ: 4
4 4
Iterations:  325
Total contacts found 1I4M: 16
16 16
Iterations:  326
Total contacts found 1I2Q: 60
60 60
Iterations:  327
Total contacts found 1ILK: 25
25 25
Iterations:  328
Total contacts found 1GPC: 56
56 56
Iterations:  329
Total contacts found 1HF9: 3
3 3
Iterations:  330
Total contacts found 1XDW: 103
103 103
Iterations:  331
Total contacts found 1E07: 208
208 208
Iterations:  332
Total contacts found 1RTQ: 73
73 73
Iterati

Total contacts found 1SR3: 36
36 36
Iterations:  471
Total contacts found 1JSF: 33
33 33
Iterations:  472
Total contacts found 1RGV: 14
14 14
Iterations:  473
Total contacts found 2CR0: 31
31 31
Iterations:  474
Total contacts found 1P0Z: 32
32 32
Iterations:  475
Total contacts found 1QLO: 4
4 4
Iterations:  476
Total contacts found 1XS3: 23
23 23
Iterations:  477
Total contacts found 2CPZ: 29
29 29
Iterations:  478
Total contacts found 2IRF: 0
0 0
Iterations:  479
Total contacts found 1ZNU: 11
11 11
Iterations:  480
Total contacts found 1F53: 57
57 57
Iterations:  481
Total contacts found 1TH5: 19
19 19
Iterations:  482
Total contacts found 1QN2: 31
31 31
Iterations:  483
Total contacts found 1BU3: 27
27 27
Iterations:  484
Total contacts found 2C3G: 23
23 23
Iterations:  485
Total contacts found 1LQB: 32
32 32
Iterations:  486
Total contacts found 1KTH: 13
13 13
Iterations:  487
Total contacts found 104L: 40
40 40
Iterations:  488
Total contacts found 1C1K: 49
49 49
Iterations:  489

Total contacts found 1FPW: 44
44 44
Iterations:  627
Total contacts found 1TNR: 37
37 37
Iterations:  628
Total contacts found 1OTF: 11
11 11
Iterations:  629
Total contacts found 1DBW: 28
28 28
Iterations:  630
Total contacts found 2A55: 24
24 24
Iterations:  631
Total contacts found 1Z6H: 31
31 31
Iterations:  632
Total contacts found 1NSD: 135
135 135
Iterations:  633
Total contacts found 1NTN: 18
18 18
Iterations:  634
Total contacts found 1FLJ: 78
78 78
Iterations:  635
Total contacts found 1KPS: 54
54 54
Iterations:  636
Total contacts found 2EZL: 14
14 14
Iterations:  637
Total contacts found 2CFE: 49
49 49
Iterations:  638
Total contacts found 1PUC: 8
8 8
Iterations:  639
Total contacts found 1C3H: 47
47 47
Iterations:  640
Total contacts found 1YUB: 72
72 72
Iterations:  641
Total contacts found 2B9D: 16
16 16
Iterations:  642
Total contacts found 1WL7: 96
96 96
Iterations:  643
Total contacts found 1IGO: 93
93 93
Iterations:  644
Total contacts found 1KCN: 2
2 2
Iterations:  

Total contacts found 1UCV: 20
20 20
Iterations:  785
Total contacts found 1X66: 18
18 18
Iterations:  786
Total contacts found 1YP5: 20
20 20
Iterations:  787
Total contacts found 1KJ6: 11
11 11
Iterations:  788
Total contacts found 1UEZ: 33
33 33
Iterations:  789
Total contacts found 1CMR: 12
12 12
Iterations:  790
Total contacts found 1IHO: 73
73 73
Iterations:  791
Total contacts found 1WFF: 13
13 13
Iterations:  792
Total contacts found 1Q5W: 9
9 9
Iterations:  793
Total contacts found 1C3Z: 20
20 20
Iterations:  794
Total contacts found 1ZH1: 46
46 46
Iterations:  795
Total contacts found 2COQ: 36
36 36
Iterations:  796
Total contacts found 1KS5: 72
72 72
Iterations:  797
Total contacts found 1HSL: 58
58 58
Iterations:  798
Total contacts found 256B: 16
16 16
Iterations:  799
Total contacts found 1EMY: 21
21 21
Iterations:  800
Total contacts found 1WUM: 28
28 28
Iterations:  801
Total contacts found 1SPF: 19
19 19
Iterations:  802
Total contacts found 1QD9: 32
32 32
Iterations:  

Total contacts found 1Y8M: 26
26 26
Iterations:  941
Total contacts found 1NTV: 39
39 39
Iterations:  942
Total contacts found 1PSF: 12
12 12
Iterations:  943
Total contacts found 1OAW: 17
17 17
Iterations:  944
Total contacts found 1WFD: 34
34 34
Iterations:  945
Total contacts found 1VZE: 73
73 73
Iterations:  946
Total contacts found 1MV5: 78
78 78
Iterations:  947
Total contacts found 1C39: 51
51 51
Iterations:  948
Total contacts found 1CBI: 42
42 42
Iterations:  949
Total contacts found 1D2S: 61
61 61
Iterations:  950
Total contacts found 1TI5: 29
29 29
Iterations:  951
Total contacts found 1A78: 47
47 47
Iterations:  952
Total contacts found 1SP7: 1
1 1
Iterations:  953
Total contacts found 1Q08: 13
13 13
Iterations:  954
Total contacts found 1SXE: 29
29 29
Iterations:  955
Total contacts found 2AX4: 45
45 45
Iterations:  956
Total contacts found 2CQD: 16
16 16
Iterations:  957
Total contacts found 1UQ5: 68
68 68
Iterations:  958
Total contacts found 1BOR: 21
21 21
Iterations:  

Total contacts found 1WHA: 27
27 27
Iterations:  1098
Total contacts found 1WH7: 21
21 21
Iterations:  1099
Total contacts found 2CRP: 38
38 38
Iterations:  1100
Total contacts found 1EDV: 4
4 4
Iterations:  1101
Total contacts found 1CKA: 19
19 19
Iterations:  1102
Total contacts found 1X8D: 27
27 27
Iterations:  1103
Total contacts found 1RDU: 38
38 38
Iterations:  1104
Total contacts found 1WHX: 24
24 24
Iterations:  1105
Total contacts found 1HCD: 39
39 39
Iterations:  1106
Total contacts found 1WF5: 36
36 36
Iterations:  1107
Total contacts found 1A0J: 78
78 78
Iterations:  1108
Total contacts found 2A4C: 30
30 30
Iterations:  1109
Total contacts found 1CHK: 67
67 67
Iterations:  1110
Total contacts found 1A92: 11
11 11
Iterations:  1111
Total contacts found 1WWN: 27
27 27
Iterations:  1112
Total contacts found 1CQK: 28
28 28
Iterations:  1113
Total contacts found 1SR2: 36
36 36
Iterations:  1114
Total contacts found 2CPH: 22
22 22
Iterations:  1115
Total contacts found 1UUN: 59
5

In [328]:
len(in_contact_sites)

1214

## Initialize Contact Dictionaries for Training Random Forest Classifier

In [329]:
# from collections import defaultdict

# # Assuming protein_structure and protein_sequence are defined

# contacts = defaultdict(dict)

# # Mapping of chain IDs to starting index
# chain_to_index = {'A': -1, 'B': 275, 'P': 375}

# count = 0


# ### Contacts from the SAME chain are only being considered

# for chain in protein_structure:
# #     print(chain.id)
#     index = chain_to_index.get(chain.id, 0)  # Default to 0 if chain ID not found

#     for residue1 in chain:
#         for residue2 in chain:
#             if residue1 != residue2:
#                 try:
#                     # Calculate Alpha-Carbon Distance
#                     distance = abs(residue1['CA'] - residue2['CA'])
#                 except KeyError:
#                     continue

#                 # Calculating Distance of Amino Acids in Sequence
#                 diff = abs(residue1.id[1] - residue2.id[1])
#                 if diff > 2:
#                     res1_index = residue1.id[1] + index
#                     res2_index = residue2.id[1] + index

#                     # Ensure indices are within the bounds of protein_sequence
#                     if 0 <= res1_index < len(protein_sequence) and 0 <= res2_index < len(protein_sequence):
#                         contact = distance < 5
#                         contacts[(res1_index, res2_index)] = {
#                             'aa1': protein_sequence[res1_index],
#                             'aa2': protein_sequence[res2_index], 
#                             'dist': distance,
#                             'contact': contact
#                         }
#                         if contact:
#                             count += 1
# #                         print(res1_index, residue1.resname, res2_index, residue2.resname, distance, protein_sequence[res1_index], protein_sequence[res2_index], contact)

# print(f"Total contacts: {count}")


In [332]:
list(in_contact_sites.items())[:1]

[('1PSM',
  [{'res_1': 14,
    'res_2': 11,
    'sig_1': 'E',
    'sig_2': 'Q',
    'dist': 4.4292865,
    'in_contact': True},
   {'res_1': 18,
    'res_2': 15,
    'sig_1': 'K',
    'sig_2': 'Q',
    'dist': 4.917969,
    'in_contact': True},
   {'res_1': 20,
    'res_2': 17,
    'sig_1': 'A',
    'sig_2': 'A',
    'dist': 4.9485846,
    'in_contact': True},
   {'res_1': 22,
    'res_2': 19,
    'sig_1': 'N',
    'sig_2': 'D',
    'dist': 4.971901,
    'in_contact': True},
   {'res_1': 25,
    'res_2': 22,
    'sig_1': 'K',
    'sig_2': 'N',
    'dist': 4.919744,
    'in_contact': True},
   {'res_1': 27,
    'res_2': 24,
    'sig_1': 'A',
    'sig_2': 'S',
    'dist': 4.610091,
    'in_contact': True},
   {'res_1': 32,
    'res_2': 29,
    'sig_1': 'K',
    'sig_2': 'E',
    'dist': 4.940884,
    'in_contact': True},
   {'res_1': 36,
    'res_2': 33,
    'sig_1': 'N',
    'sig_2': 'E',
    'dist': 4.714307,
    'in_contact': True}])]

In [333]:
list(subset_non_contact_sites.items())[:1]

[('1PSM',
  [{'res_1': 30,
    'res_2': 12,
    'sig_1': 'A',
    'sig_2': 'D',
    'dist': 30.428394,
    'in_contact': False},
   {'res_1': 17,
    'res_2': 4,
    'sig_1': 'A',
    'sig_2': 'K',
    'dist': 22.65882,
    'in_contact': False},
   {'res_1': 34,
    'res_2': 4,
    'sig_1': 'A',
    'sig_2': 'K',
    'dist': 42.85483,
    'in_contact': False},
   {'res_1': 30,
    'res_2': 2,
    'sig_1': 'A',
    'sig_2': 'A',
    'dist': 43.430656,
    'in_contact': False},
   {'res_1': 37,
    'res_2': 13,
    'sig_1': 'L',
    'sig_2': 'A',
    'dist': 28.255463,
    'in_contact': False},
   {'res_1': 33,
    'res_2': 5,
    'sig_1': 'E',
    'sig_2': 'K',
    'dist': 38.63465,
    'in_contact': False},
   {'res_1': 19,
    'res_2': 16,
    'sig_1': 'D',
    'sig_2': 'A',
    'dist': 5.5796056,
    'in_contact': False},
   {'res_1': 28,
    'res_2': 17,
    'sig_1': 'E',
    'sig_2': 'A',
    'dist': 16.961533,
    'in_contact': False}])]

In [368]:
from collections import defaultdict

# Initialize contact_data as a defaultdict of lists
contact_data = defaultdict(list)

# Add data from in_contact_sites
for pdb_id, contacts in in_contact_sites.items():
    for contact in contacts:
        contact_data[pdb_id].append({
            'res_1': contact['res_1'],
            'res_2': contact['res_2'],
            'sig_1': contact['sig_1'],
            'sig_2': contact['sig_2'],
            'dist': contact['dist'],
            'in_contact': contact['in_contact']
        })

# Add data from subset_non_contact_sites
for pdb_id, non_contacts in subset_non_contact_sites.items():
    for non_contact in non_contacts:
        contact_data[pdb_id].append({
            'res_1': non_contact['res_1'],
            'res_2': non_contact['res_2'],
            'sig_1': non_contact['sig_1'],
            'sig_2': non_contact['sig_2'],
            'dist': non_contact['dist'],
            'in_contact': non_contact['in_contact']
        })

# contact_data is now a defaultdict containing all the data from both dictionaries


In [377]:
from sklearn.model_selection import train_test_split

# Assuming contact_data is defined and is a list of dictionaries
total_data = contact_data

# First split: Splitting into 80% for training and 20% for the remaining
train_data, remaining_data = train_test_split(total_data, test_size=0.20, random_state=42)

# Second split: Splitting the remaining 20% into 15% test and 5% validation of the original data
# This corresponds to 75% (test) and 25% (validation) of the remaining 20% data
test_data, validation_data = train_test_split(remaining_data, test_size=0.25, random_state=42)

# Print the sizes to verify
print(f"Training set size: {len(train_data)}")
print(f"Test set size: {len(test_data)}")
print(f"Validation set size: {len(validation_data)}")


Training set size: 953
Test set size: 179
Validation set size: 60


## Generate ESM-2 Embedding

In [340]:
# 1. Load the ESM Model
model, alphabet = esm.pretrained.esm2_t33_650M_UR50D()
model.eval()

ESM2(
  (embed_tokens): Embedding(33, 1280, padding_idx=1)
  (layers): ModuleList(
    (0-32): 33 x TransformerLayer(
      (self_attn): MultiheadAttention(
        (k_proj): Linear(in_features=1280, out_features=1280, bias=True)
        (v_proj): Linear(in_features=1280, out_features=1280, bias=True)
        (q_proj): Linear(in_features=1280, out_features=1280, bias=True)
        (out_proj): Linear(in_features=1280, out_features=1280, bias=True)
        (rot_emb): RotaryEmbedding()
      )
      (self_attn_layer_norm): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
      (fc1): Linear(in_features=1280, out_features=5120, bias=True)
      (fc2): Linear(in_features=5120, out_features=1280, bias=True)
      (final_layer_norm): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
    )
  )
  (contact_head): ContactPredictionHead(
    (regression): Linear(in_features=660, out_features=1, bias=True)
    (activation): Sigmoid()
  )
  (emb_layer_norm_after): LayerNorm((1280,), eps=1

In [341]:
# Set model to use cuda GPU
if torch.cuda.is_available():
    model = model.cuda()

In [349]:
# Index protein sequence as sequence 0 (next sequence would be indexed as 1)

def generate_embeddings(pdb_id):
    protein_sequence = protein_data[pdb_id]
    esm_input_data = [(0, protein_sequence)]
    # print('Data: ', esm_input_data, '\n')

    # Prepare variables to input sequence into ESM-2 model 
    batch_converter = alphabet.get_batch_converter()
    batch_labels, batch_strs, batch_tokens = batch_converter(esm_input_data)
    batch_tokens = batch_tokens.cuda() if torch.cuda.is_available() else batch_tokens

    # print('batch_tokens: ', '\n\n', batch_tokens, '\n')

    # 4. Input prepared sequence information into model and output as results (contact predictions are included in embedding)
    with torch.no_grad():
        results = model(batch_tokens, repr_layers=[33], return_contacts=True)

    return results['attentions']

In [356]:
attention_data = {}

for pdb_id in same_sequence_ids[:50]:
    attention_data[pdb_id] = generate_embeddings(pdb_id)

In [353]:
len(attention_data)

47

## Implementing Random Forest Classifier (Currently for only 1 sequence)

In [354]:
from sklearn.ensemble import RandomForestClassifier

In [364]:
contact_data[0]

{'id': '1PSM',
 'res_1': 14,
 'res_2': 11,
 'sig_1': 'E',
 'sig_2': 'Q',
 'dist': 4.4292865,
 'in_contact': True}

In [355]:
# Extract attentions from all heads and layers for given amino acid residues

def get_x_y(pdb_id, res_1, res_2):
    vectors = []
    in_contact = contact_data[pdb_id]['res_1']
    for layer in range(0,33):
        for head in range(0,20):
            vectors.append(results['attentions'][0][layer][head][res1][res2])

    return vectors, in_contact

vectors, in_contact = get_x_y(1,103)
print(in_contact)
print(vectors[:5])

TypeError: list indices must be integers or slices, not tuple

In [ ]:
# Set X to extracted attentions from layers and heads
# Set Y to True/False if the amino acids are in contact

X = []
y = []

for i in list(contacts.keys())[:1500]:
    vectors, in_contact = get_x_y(i[0],i[1])
    X.append(vectors)
    y.append(in_contact)
    
X = np.array(X)

In [ ]:
# Fit classifier

clf = RandomForestClassifier(max_depth=2, random_state=0)
clf.fit(X, y)

In [ ]:
## Find X vectors that are supposed to be True

# count = 0

# for vec,cont in zip(X,y):
#     count+=1
#     if cont == True:
#         print(count)
#         print(vec)

In [ ]:
contacts[(1,103)]

In [ ]:
# Testing prediction on known True contact

vectors, in_contact = get_x_y(1, 103)
# vectors = X[372]
vectors = np.array(vectors).reshape(1, -1)  # Reshape for a single sample prediction
prediction = clf.predict(vectors)
prediction